# Lab 3
## Beamforming

# Part 1 
Here initially study narrowband beamforming and how investigate how different parameters influence the output of the beamformer. We build a class which will implement the beamformer to a narrowband signal which will be used as our base class in which we will be able to compare this against our later study in this lab. Our class assumes a Uniform Linear Array (ULA) with angles in radians and spacing in wavelengths.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class NarrowbandBeamformer:
    def __init__(self, n_elements, d=0.5, wavelength=1.0, window="rect"):
        self.N = n_elements
        self.d = d
        self.wavelength = wavelength
        self.k = 2*np.pi / self.wavelength
        self.n = np.arange(self.N)

        self.window = self._make_window(window)
        self.theta_steer = 0.0
        self.w = None
        self.steer_conventional(self.theta_steer)

    def _make_window(self,window):
        if isinstance(window,str):
            wtype = window.lower()
            if wtype == "rect":
                w = np.ones(self.N)
            elif wtype == "hann":
                w = np.hanning(self.N)
            elif wtype == "hamming":
                w = np.hamming(self.N)
            else:
                raise ValueError(f"Unknown Window type")    
        else:
            w = np.asarray(window)
            if w.shape[0] != self.N:
                raise ValueError(f"Window must have the correct length")
        return w/np.max(np.abs(w))
    def update_weights(self, theta):
        a = self.steering_vector(self.theta_steer)  
        self.w = self.window * a
    def steering_vector(self, theta):
        theta = np.asarray(theta)
        phase = self.k*self.d*np.outer(self.n, np.sin(theta))
        a = np.exp(1j*phase)
        return a   
    def steer_convent(self, theta):
        self.theta_steer = theta
        self.update_weights()
    def beamform(self, X):
        X = np.asarray(X)
        return np.conj(self.w)@X
    def beampattern(self, theta_grid):
        theta_grid = np.asarray(theta_grid)
        a = self.steering_vector(theta_grid)
        B = np.conj(self.w)@a
        P = np.abs(B)**2

        P/=np.ax(P)
        P = 10*np.log10(np.maximum(P,1e-12))
        return P
    def doa_scan(self, X, theta_scan):
        X = np.asarray(X)
        theta_scan = np.asarray(theta_scan)
        metric = np.zeros_like(theta_scan)
        for i, th in enumerate(theta_scan):
            self.steer_convent(th)
            y = self.beamform(X)
            metric[i] = np.var(y)
        theta_hat = theta_scan[np.argmax(metric)]
        return theta_hat, metric
    




